In [3]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [4]:
img_rows=28
img_cols=28
channels=1
img_shape=(img_rows, img_cols, channels)

In [10]:
def build_generator():
    noise_shape=(100,)
    
    model=Sequential()
    
    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise=Input(shape=noise_shape)
    img=model(noise)
    
    return Model(noise, img)

In [11]:
def build_discriminator():
    
    model=Sequential()
    
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    img=Input(shape=img_shape)
    validity=model(img)
    
    return Model(img, validity)

In [12]:
def train(epochs, batch_size=128, save_interval=500):
    (X_train, _), (_, _)=mnist.load_data() #load real images of unisigned intgeers
    X_train=(X_train.astype(np.float32)-127.5)/127.5 #rescale them between -1 to 1
    X_train=np.expand_dims(X_train, axis=3)
    half_batch=int(batch_size/2) #half of gen and real imgs go to discriminator
    
    for epoch in range(epochs):
        idx=np.random.randint(0, X_train.shape[0], half_batch) #take random half batch
        imgs=X_train[idx]
        
        noise=np.random.normal(0, 1, (half_batch, 100)) #100 dimensional noise values
        
        gen_imgs=generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1))) #y value set to 1 for real images
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1))) #y value set to 0 for fake images
        
        d_loss = 0.5*np.add(d_loss_real, d_loss_fake) #avg of real and fake
        #train generator
        noise=np.random.normal(0, 1, (batch_size, 100))
        
        valid_y = np.array([1]*batch_size) #generates an array of all 1s in column format
        
        g_loss = combined.train_on_batch(noise, valid_y)
        
        print("%d [D loss: %f, acc.: %.2f%%][G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [22]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r*c, 100)) #generate 25 images of the saved images
    gen_imgs = generator.predict(noise)
    
    gen_imgs = .5*gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :, :,0], cmap='gray')
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [15]:
optimizer= Adam(0.0002, 0.5)

discriminator=build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#binary crossentropy works better than mean cross error 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

In [16]:
generator=build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_8 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [17]:
z=Input(shape=(100,))
img=generator(z)

In [18]:
discriminator.trainable=False

In [19]:
valid=discriminator(img)

In [20]:
combined=Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [29]:
train(epochs=10000, batch_size=32, save_interval=1000)

1/1 [==============================] - 0s 29ms/step
0 [D loss: 0.675275, acc.: 59.38%][G loss: 0.798418]
1/1 [==============================] - 0s 26ms/step
1 [D loss: 0.606646, acc.: 71.88%][G loss: 0.829631]
1/1 [==============================] - 0s 26ms/step
2 [D loss: 0.664728, acc.: 59.38%][G loss: 0.803556]
1/1 [==============================] - 0s 26ms/step
3 [D loss: 0.693800, acc.: 56.25%][G loss: 0.770605]
1/1 [==============================] - 0s 27ms/step
4 [D loss: 0.673011, acc.: 50.00%][G loss: 0.835881]
1/1 [==============================] - 0s 30ms/step
5 [D loss: 0.583446, acc.: 68.75%][G loss: 0.838285]
1/1 [==============================] - 0s 27ms/step
6 [D loss: 0.621484, acc.: 71.88%][G loss: 0.770013]
1/1 [==============================] - 0s 21ms/step
7 [D loss: 0.704833, acc.: 46.88%][G loss: 0.725255]
1/1 [==============================] - 0s 21ms/step
8 [D loss: 0.642108, acc.: 65.62%][G loss: 0.731000]
1/1 [==============================] - 0s 21ms/step
9 [

In [30]:
generator.save("generator_model_test.h5")